# Ejercicio

In [4]:
from numpy.random import seed, choice
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
import numpy as np
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd

In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv')
pd.set_option('display.max_columns', None)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


No hacemos análisis porque es el mismo dataset que el modulo anterior

División

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Outcome', axis=1), df.Outcome, test_size=0.2, random_state=42)

In [34]:
# Convierto a array de numpy sino me da error a la hora de acceder a los índices
X_train = X_train.values
y_train = y_train.values

In [35]:
class RandomForestCustom:

  def __init__(self, n_estimators, random_state, max_depth, min_samples_leaf, max_features, X, y):
    self.n_estimators = n_estimators
    self.random_state = random_state
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf
    self.max_features = max_features
    self.X = X
    self.y = y
    self.models = []

  def get_bootstrap_dataset(self):
    seed(self.random_state)
    ls_idx_features, ls_data = [], []
    # Para cada uno de los estimadores
    for i in range(self.n_estimators):
      # Debo de seleccionar filas aleatorias de X (el dataset debe tener el mismo tamaño que el original)
      idx_rows_tree = choice(len(self.X), len(self.X), replace=True)
      # Debo de seleccionar tantas características como max_features tengo permitido
      idx_cols_tree = choice(self.X.shape[1], self.max_features, replace=False)
      # Guardar los indices seleccionados de las caracteríticas en una lista
      ls_idx_features.append(idx_cols_tree)
      # Guardar los datos seleccionados que se usarán para entrenar cada árbol
      ls_data.append((self.X[idx_rows_tree,:][:,idx_cols_tree],self.y[idx_rows_tree]))
    # Devuelvo las listas
    return ls_idx_features, ls_data

  def fit(self):
    # Obtener los datasets generados por bootstrap
    ls_idx_features, ls_data = self.get_bootstrap_dataset()
    # Para cada dataset
    for i, data in enumerate(ls_data):
      # Desempaco las X y la y de data
      X, y = data
      # Entrenar un árbol de decisión con esa versión del dataset y la configuración porporcionada por el usuario
      tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X, y)
      # Almaceno en el atributo modelos las características usadas junto con el árbol entrenado
      self.models.append((ls_idx_features[i], tree))

  def predict(self, X):
    ls_predictions = []
    # Para cada tupla del atributo models
    for idxs, tree in self.models:
      # Selecciona las columnas X que necesito para predecir
      X_to_predict = X[:,idxs]
      # Aplico el árbol entrenado sobre los datos seleccionados
      yhats = tree.predict(X_to_predict)
      # Guadamos en una lista las predicciones reportadas por el árbol
      ls_predictions.append(yhats)
    predictions = np.vstack(ls_predictions)
    final_predictions = [Counter(predictions[:,i]).most_common(1)[0][0] for i in range(predictions.shape[1])]
    return final_predictions

In [36]:
rf_simple = RandomForestCustom(n_estimators=5, random_state=42, max_depth=2, min_samples_leaf=20, max_features=X_train.shape[1]//2, X=X_train, y=y_train)
rf_simple.fit()
rf_complex = RandomForestCustom(n_estimators=100, random_state=42, max_depth=20, min_samples_leaf=1, max_features=int(X_train.shape[1]*0.8), X=X_train, y=y_train)
rf_complex.fit()

In [37]:
y_pred_train_simple = rf_simple.predict(X_train)
y_pred_test_simple = rf_simple.predict(X_test)

y_pred_train_complex = rf_complex.predict(X_train)
y_pred_test_complex = rf_complex.predict(X_test)

InvalidIndexError: (slice(None, None, None), array([7, 6, 2, 1], dtype=int32))